In [110]:
import os
import sys
import pickle

current_dir =  os.getcwd()
#parent_dir = os.path.dirname(current_dir)
#sys.path.insert(0, parent_dir)

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from cities.utils.cleaning_utils import standardize_and_scale
from cities.utils.clean_variable import communities_tracts_to_counties

import matplotlib.pyplot as plt



from cities.utils.data_grabber import DataGrabber, find_repo_root
data = DataGrabber()
data.get_features_wide(["gdp", "population", "transport",
            "spending_transportation", "spending_commerce", "spending_HHS"])
gdp = data.wide['gdp']
# population = data.wide['population']
# trnasport = data.wide['transport']
# spending_transportation = data.wide['spending_transportation']
# spending_commerce = data.wide['spending_commerce']

def tableInfo(tableName):

    print(tableName.head())
    print(tableName.dtypes)
    print(f'Number of rows: {tableName.shape[0]}')
    print(f'Unique FIPS numbers {tableName['GeoFIPS'].nunique()}')


root = find_repo_root()

In [105]:
communities = pd.read_csv(f"{root}/data/raw/communities_raw.csv")

In [164]:
print(communities.columns)

Index(['Census tract 2010 ID', 'Total population',
       'Adjusted percent of individuals below 200% Federal Poverty Line',
       'Expected agricultural loss rate (Natural Hazards Risk Index)',
       'Expected building loss rate (Natural Hazards Risk Index)',
       'Expected population loss rate (Natural Hazards Risk Index)',
       'Share of properties at risk of flood in 30 years',
       'Share of properties at risk of fire in 30 years', 'Energy burden',
       'PM2.5 in the air', 'Diesel particulate matter exposure',
       'Traffic proximity and volume', 'Housing burden (percent)',
       'Percent pre-1960s housing (lead paint indicator)',
       'Median value ($) of owner-occupied housing units',
       'Proximity to hazardous waste sites',
       'Proximity to Risk Management Plan (RMP) facilities',
       'Wastewater discharge',
       'Current asthma among adults aged greater than or equal to 18 years',
       'Diagnosed diabetes among adults aged greater than or equal to 

In [ ]:
list_variables = ['Housing burden (percent)', 'Energy burden']

burdens = communities_tracts_to_counties(communities, list_variables)

In [ ]:
nan_counts = burdens.isnull().sum()
print(nan_counts)

In [172]:
list_variables = ['Housing burden (percent)', 'Energy burden']

burdens = communities_tracts_to_counties(communities, list_variables)

In [173]:
nan_counts = burdens.isnull().sum()
print(nan_counts)

GeoFIPS                      0
Housing burden (percent)    16
Energy burden               17
dtype: int64


In [174]:
burdens

,GeoFIPS,Housing burden (percent),Energy burden
0,1001,19.573962,2.854605
1,1003,17.984645,2.522436
2,1005,22.277592,4.771224
3,1007,19.042102,5.209532
4,1009,15.335258,3.871431
...,...,...,...
3229,72151,19.673542,4.650736
3230,72153,13.587304,4.838884
3231,78010,NaN,NaN
3232,78020,NaN,NaN


In [175]:
burdens['GeoFIPS'] = burdens['GeoFIPS'].astype(np.int64) 

common_fips = np.intersect1d(burdens['GeoFIPS'].unique(), gdp['GeoFIPS'].unique())
burdens = burdens[burdens['GeoFIPS'].isin(common_fips)]
burdens = burdens.merge(gdp[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left')


burdens = burdens[['GeoFIPS', 'GeoName', 'Housing burden (percent)', 'Energy burden']]

burdens.columns = ['GeoFIPS', 'GeoName', 'Housing_burden', 'Energy_burden']

columns_to_trans = burdens.columns[-2:]
burdens[columns_to_trans] = burdens[columns_to_trans].astype('float64')
    

In [176]:
burdens

,GeoFIPS,GeoName,Housing_burden,Energy_burden
0,1001,"Autauga, AL",19.573962,2.854605
1,1003,"Baldwin, AL",17.984645,2.522436
2,1005,"Barbour, AL",22.277592,4.771224
3,1007,"Bibb, AL",19.042102,5.209532
4,1009,"Blount, AL",15.335258,3.871431
...,...,...,...,...
3067,56037,"Sweetwater, WY",15.536775,1.750052
3068,56039,"Teton, WY",17.488574,1.460009
3069,56041,"Uinta, WY",15.547195,2.302066
3070,56043,"Washakie, WY",15.334870,3.307711


In [153]:
list_variables = ['Expected agricultural loss rate (Natural Hazards Risk Index)',
       'Expected building loss rate (Natural Hazards Risk Index)',
       'Expected population loss rate (Natural Hazards Risk Index)',
     'Diesel particulate matter exposure',
       'Proximity to hazardous waste sites',
       'Proximity to Risk Management Plan (RMP) facilities',
     ]

hazards_polution = communities_tracts_to_counties(communities, list_variables)

In [154]:
nan_counts = hazards_polution.isnull().sum()
print(nan_counts)



GeoFIPS                                                          0
Expected agricultural loss rate (Natural Hazards Risk Index)    94
Expected building loss rate (Natural Hazards Risk Index)        94
Expected population loss rate (Natural Hazards Risk Index)      94
Diesel particulate matter exposure                              16
Proximity to hazardous waste sites                              16
Proximity to Risk Management Plan (RMP) facilities              16
dtype: int64


In [155]:
hazards_polution

,GeoFIPS,Expected agricultural loss rate (Natural Hazards Risk Index),Expected building loss rate (Natural Hazards Risk Index),Expected population loss rate (Natural Hazards Risk Index),Diesel particulate matter exposure,Proximity to hazardous waste sites,Proximity to Risk Management Plan (RMP) facilities
0,1001,0.302055,0.019612,0.000257,0.191407,0.681768,0.087971
1,1003,0.459665,0.021760,0.000163,0.154058,0.372676,0.114923
2,1005,1.657605,0.015929,0.000485,0.081227,0.022192,0.279859
3,1007,0.620461,0.033424,0.000600,0.100779,0.282008,0.033311
4,1009,0.436726,0.033897,0.000585,0.134566,0.170742,0.080232
...,...,...,...,...,...,...,...
3229,72151,NaN,NaN,NaN,0.015640,0.257403,1.110156
3230,72153,NaN,NaN,NaN,0.011302,1.317716,0.837912
3231,78010,NaN,NaN,NaN,NaN,NaN,NaN
3232,78020,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
hazard = hazards_polution.copy()

hazard.dropna(inplace=True)

In [157]:
hazard

,GeoFIPS,Expected agricultural loss rate (Natural Hazards Risk Index),Expected building loss rate (Natural Hazards Risk Index),Expected population loss rate (Natural Hazards Risk Index),Diesel particulate matter exposure,Proximity to hazardous waste sites,Proximity to Risk Management Plan (RMP) facilities
0,1001,0.302055,0.019612,0.000257,0.191407,0.681768,0.087971
1,1003,0.459665,0.021760,0.000163,0.154058,0.372676,0.114923
2,1005,1.657605,0.015929,0.000485,0.081227,0.022192,0.279859
3,1007,0.620461,0.033424,0.000600,0.100779,0.282008,0.033311
4,1009,0.436726,0.033897,0.000585,0.134566,0.170742,0.080232
...,...,...,...,...,...,...,...
3138,56037,0.127733,0.001405,0.000075,0.139012,0.803674,0.325963
3139,56039,0.195065,0.027654,0.008988,0.022185,0.000000,0.010000
3140,56041,0.727953,0.026565,0.000200,0.054165,0.054491,0.585652
3141,56043,0.574732,0.008747,0.000138,0.031689,0.000000,0.123816


In [158]:
# do it with health

hazard['GeoFIPS'] = hazard['GeoFIPS'].astype(np.int64) 

common_fips = np.intersect1d(hazard['GeoFIPS'].unique(), health['GeoFIPS'].unique())
hazard = hazard[hazard['GeoFIPS'].isin(common_fips)]
hazard = hazard.merge(health[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left')

In [159]:
hazard

,GeoFIPS,Expected agricultural loss rate (Natural Hazards Risk Index),Expected building loss rate (Natural Hazards Risk Index),Expected population loss rate (Natural Hazards Risk Index),Diesel particulate matter exposure,Proximity to hazardous waste sites,Proximity to Risk Management Plan (RMP) facilities,GeoName
0,1001,0.302055,0.019612,0.000257,0.191407,0.681768,0.087971,"Autauga, AL"
1,1003,0.459665,0.021760,0.000163,0.154058,0.372676,0.114923,"Baldwin, AL"
2,1005,1.657605,0.015929,0.000485,0.081227,0.022192,0.279859,"Barbour, AL"
3,1007,0.620461,0.033424,0.000600,0.100779,0.282008,0.033311,"Bibb, AL"
4,1009,0.436726,0.033897,0.000585,0.134566,0.170742,0.080232,"Blount, AL"
...,...,...,...,...,...,...,...,...
3016,56037,0.127733,0.001405,0.000075,0.139012,0.803674,0.325963,"Sweetwater, WY"
3017,56039,0.195065,0.027654,0.008988,0.022185,0.000000,0.010000,"Teton, WY"
3018,56041,0.727953,0.026565,0.000200,0.054165,0.054491,0.585652,"Uinta, WY"
3019,56043,0.574732,0.008747,0.000138,0.031689,0.000000,0.123816,"Washakie, WY"


In [161]:
hazard = hazard[['GeoFIPS', 'GeoName', 'Expected agricultural loss rate (Natural Hazards Risk Index)',
       'Expected building loss rate (Natural Hazards Risk Index)',
       'Expected population loss rate (Natural Hazards Risk Index)',
     'Diesel particulate matter exposure',
       'Proximity to hazardous waste sites',
       'Proximity to Risk Management Plan (RMP) facilities',]]

hazard.columns = ['GeoFIPS', 'GeoName', 'Expected_agricultural_loss_rate', 'Expected_building_loss_rate', 'Expected_population_loss_rate', 'Diesel_particulate_matter_exposure', 'Proximity_to_hazardous_waste_sites', 'Proximity_to_Risk_Management_Plan_facilities']

In [162]:
columns_to_trans = hazard.columns[-6:]
hazard[columns_to_trans] = hazard[columns_to_trans].astype('float64')


In [55]:
column_data_types = {'column1': str, 'column18': str, 'column26': str, 'column70': str, 'column72': str, 'column85': str, 'column131': str}
communities = pd.read_csv(f"{root}/data/raw/1.0-communities.csv", dtype=column_data_types, low_memory=False)
communities.head()

,Census tract 2010 ID,County Name,State/Territory,Percent Black or African American alone,Percent American Indian / Alaska Native,Percent Asian,Percent Native Hawaiian or Pacific,Percent two or more races,Percent White,Percent Hispanic or Latino,...,Percent individuals age 25 or over with less than high school degree,Percent of residents who are not currently enrolled in higher ed,Unemployment (percent) in 2009 (island areas) and 2010 (states and PR),Percentage households below 100% of federal poverty line in 2009 (island areas) and 2010 (states and PR),Greater than or equal to the 90th percentile for unemployment and has low HS education in 2009 (island areas)?,Greater than or equal to the 90th percentile for households at or below 100% federal poverty level and has low HS education in 2009 (island areas)?,Greater than or equal to the 90th percentile for low median household income as a percent of area median income and has low HS education in 2009 (island areas)?,Number of Tribal areas within Census tract for Alaska,Names of Tribal areas within Census tract,Percent of the Census tract that is within Tribal areas
0,1001020100,Autauga County,Alabama,0.07,0.0,0.00,0.00,0.07,0.83,0.01,...,12.0,91.0,2.0,10.0,False,False,False,NaN,NaN,NaN
1,1001020200,Autauga County,Alabama,0.57,0.0,0.00,0.01,0.02,0.38,0.01,...,14.0,96.0,5.0,14.0,False,False,False,NaN,NaN,NaN
2,1001020300,Autauga County,Alabama,0.24,0.0,0.00,0.00,0.02,0.65,0.06,...,14.0,97.0,5.0,8.0,False,False,False,NaN,NaN,NaN
3,1001020400,Autauga County,Alabama,0.05,0.0,0.00,0.00,0.03,0.89,0.01,...,8.0,94.0,3.0,6.0,False,False,False,NaN,NaN,NaN
4,1001020500,Autauga County,Alabama,0.18,0.0,0.03,0.00,0.02,0.70,0.04,...,7.0,94.0,2.0,5.0,False,False,False,NaN,NaN,NaN


In [56]:
communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].astype(str)

communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].astype(str).str.zfill(11)
communities.head(2)

,Census tract 2010 ID,County Name,State/Territory,Percent Black or African American alone,Percent American Indian / Alaska Native,Percent Asian,Percent Native Hawaiian or Pacific,Percent two or more races,Percent White,Percent Hispanic or Latino,...,Percent individuals age 25 or over with less than high school degree,Percent of residents who are not currently enrolled in higher ed,Unemployment (percent) in 2009 (island areas) and 2010 (states and PR),Percentage households below 100% of federal poverty line in 2009 (island areas) and 2010 (states and PR),Greater than or equal to the 90th percentile for unemployment and has low HS education in 2009 (island areas)?,Greater than or equal to the 90th percentile for households at or below 100% federal poverty level and has low HS education in 2009 (island areas)?,Greater than or equal to the 90th percentile for low median household income as a percent of area median income and has low HS education in 2009 (island areas)?,Number of Tribal areas within Census tract for Alaska,Names of Tribal areas within Census tract,Percent of the Census tract that is within Tribal areas
0,01001020100,Autauga County,Alabama,0.07,0.0,0.0,0.00,0.07,0.83,0.01,...,12.0,91.0,2.0,10.0,False,False,False,NaN,NaN,NaN
1,01001020200,Autauga County,Alabama,0.57,0.0,0.0,0.01,0.02,0.38,0.01,...,14.0,96.0,5.0,14.0,False,False,False,NaN,NaN,NaN


In [57]:
communities['GeoFIPS'] = communities['Census tract 2010 ID'].astype(str).str[:5]

In [58]:
print(communities['GeoFIPS'])

print(communities['GeoFIPS'].nunique())

print(communities.shape[0])




0        01001
1        01001
2        01001
3        01001
4        01001
         ...  
74129    78030
74130    78030
74131    78030
74132    78030
74133    78030
Name: GeoFIPS, Length: 74134, dtype: object
3234
74134


In [59]:
communities.iloc[:, 22]


0         1993.0
1         1959.0
2         3507.0
3         3878.0
4        10596.0
          ...   
74129     4878.0
74130     5220.0
74131     4356.0
74132     2816.0
74133        NaN
Name: Total population, Length: 74134, dtype: float64

In [60]:


columns = [0, 22, 24, 29, 32, 35, 37, 41, 46, 49, 52, 55, 60,
63, 65, 77, 83, 92, 98, 101, 104, 107, 119, 121, 123]

subset = communities.iloc[:, columns]
subset['GeoFIPS'] = communities['GeoFIPS']

subset


C:\Users\nikod\AppData\Local\Temp\ipykernel_14852\833122963.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['GeoFIPS'] = communities['GeoFIPS']


,Census tract 2010 ID,Total population,Adjusted percent of individuals below 200% Federal Poverty Line,Expected agricultural loss rate (Natural Hazards Risk Index),Expected building loss rate (Natural Hazards Risk Index),Expected population loss rate (Natural Hazards Risk Index),Share of properties at risk of flood in 30 years,Share of properties at risk of fire in 30 years,Energy burden,PM2.5 in the air,...,Proximity to Risk Management Plan (RMP) facilities,Wastewater discharge,Current asthma among adults aged greater than or equal to 18 years,Diagnosed diabetes among adults aged greater than or equal to 18 years,Coronary heart disease among adults aged greater than or equal to 18 years,Life expectancy (years),Percent of individuals below 200% Federal Poverty Line,Percent of individuals < 100% Federal Poverty Line,Percent individuals age 25 or over with less than high school degree,GeoFIPS
0,01001020100,1993.0,0.18,0.0776,0.0253,0.0004,22.0,67.0,2.0,9.15,...,0.07,NaN,990.0,1130.0,640.0,73.09,27.0,16.0,12.0,01001
1,01001020200,1959.0,0.40,0.0053,0.0278,0.0003,25.0,23.0,1.0,9.18,...,0.08,NaN,1100.0,1420.0,590.0,76.90,43.0,19.0,14.0,01001
2,01001020300,3507.0,0.26,0.0527,0.0163,0.0002,7.0,32.0,3.0,9.20,...,0.08,NaN,1019.0,1180.0,650.0,NaN,29.0,17.0,14.0,01001
3,01001020400,3878.0,0.16,0.0698,0.0174,0.0002,9.0,29.0,3.0,9.23,...,0.08,NaN,880.0,1080.0,680.0,75.40,21.0,4.0,8.0,01001
4,01001020500,10596.0,0.14,0.1419,0.0171,0.0002,3.0,5.0,2.0,9.24,...,0.09,0.0,919.0,919.0,500.0,79.40,20.0,15.0,7.0,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74129,78030960900,4878.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78030
74130,78030961000,5220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78030
74131,78030961100,4356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78030
74132,78030961200,2816.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78030


In [61]:
subset.columns

Index(['Census tract 2010 ID', 'Total population',
       'Adjusted percent of individuals below 200% Federal Poverty Line',
       'Expected agricultural loss rate (Natural Hazards Risk Index)',
       'Expected building loss rate (Natural Hazards Risk Index)',
       'Expected population loss rate (Natural Hazards Risk Index)',
       'Share of properties at risk of flood in 30 years',
       'Share of properties at risk of fire in 30 years', 'Energy burden',
       'PM2.5 in the air', 'Diesel particulate matter exposure',
       'Traffic proximity and volume', 'Housing burden (percent)',
       'Percent pre-1960s housing (lead paint indicator)',
       'Median value ($) of owner-occupied housing units',
       'Proximity to hazardous waste sites',
       'Proximity to Risk Management Plan (RMP) facilities',
       'Wastewater discharge',
       'Current asthma among adults aged greater than or equal to 18 years',
       'Diagnosed diabetes among adults aged greater than or equal to 

In [62]:
def weighted_mean(group, column):
    values = group[column]
    weights = group['Total population']
    
    not_nan_indices = ~np.isnan(values)

    if np.any(not_nan_indices) and np.sum(weights[not_nan_indices]) != 0:
        weighted_values = values[not_nan_indices] * weights[not_nan_indices]
        return np.sum(weighted_values) / np.sum(weights[not_nan_indices])
    else:
        return np.nan
    
    
    
def agregate_tracts_to_counties(data, list_variables)-> pd.DataFrame:  # using the weighted mean function for total population
    
    all_results = pd.DataFrame()

    for variable in list_variables:
        weighted_avg = subset.groupby('GeoFIPS').apply(weighted_mean, column=variable).reset_index()
        weighted_avg.columns = ['GeoFIPS', variable]

        nan_counties = subset.groupby('GeoFIPS').apply(lambda x: all(np.isnan(x[variable]))).reset_index()
        nan_counties.columns = ['GeoFIPS', 'all_nan']

        result_df = pd.merge(weighted_avg, nan_counties, on='GeoFIPS')
        result_df.loc[result_df['all_nan'], variable] = np.nan

        result_df = result_df.drop(columns=['all_nan'])

        if 'GeoFIPS' not in all_results.columns:
            all_results = result_df.copy()
        else:
            all_results = pd.merge(all_results, result_df, on='GeoFIPS', how='left')

    return all_results




,GeoFIPS,GeoName,LifeExpectancy,Asthma,Diabetes,HeartDisease
0,1001,"Autauga, AL",76.13,9.84,11.53,6.24
1,1003,"Baldwin, AL",77.92,8.93,11.01,6.91
2,1005,"Barbour, AL",74.12,11.20,17.68,9.22
3,1007,"Bibb, AL",73.95,10.03,13.18,7.37
4,1009,"Blount, AL",76.06,9.87,12.29,7.71
...,...,...,...,...,...,...
3016,56037,"Sweetwater, WY",79.19,8.99,7.39,4.54
3017,56039,"Teton, WY",83.02,8.15,6.02,3.92
3018,56041,"Uinta, WY",77.62,9.69,8.09,5.50
3019,56043,"Washakie, WY",80.56,9.25,10.41,7.06


In [93]:
list_variables = ['Life expectancy (years)', 'Current asthma among adults aged greater than or equal to 18 years',
                    'Diagnosed diabetes among adults aged greater than or equal to 18 years', 'Coronary heart disease among adults aged greater than or equal to 18 years']

health = agregate_tracts_to_counties(subset, list_variables)

In [94]:
health.dropna(inplace=True)



health.head()


,GeoFIPS,Life expectancy (years),Current asthma among adults aged greater than or equal to 18 years,Diagnosed diabetes among adults aged greater than or equal to 18 years,Coronary heart disease among adults aged greater than or equal to 18 years
0,01001,76.130687,984.005218,1152.975316,623.823041
1,01003,77.916883,892.861584,1100.995301,690.749185
2,01005,74.117111,1119.739364,1767.607310,922.343717
3,01007,73.945111,1002.516783,1318.238119,737.272929
4,01009,76.057294,987.407916,1229.246892,770.943638


In [95]:
health['GeoFIPS'] = health['GeoFIPS'].astype(np.int64) 

common_fips = np.intersect1d(health['GeoFIPS'].unique(), gdp['GeoFIPS'].unique())
health = health[health['GeoFIPS'].isin(common_fips)]
health = health.merge(gdp[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left')


In [96]:

health = health[['GeoFIPS', 'GeoName', 'Life expectancy (years)', 'Current asthma among adults aged greater than or equal to 18 years',
                'Diagnosed diabetes among adults aged greater than or equal to 18 years', 'Coronary heart disease among adults aged greater than or equal to 18 years']]

health.columns = ['GeoFIPS', 'GeoName', 'LifeExpectancy', 'Asthma', 'Diabetes', 'HeartDisease']


In [97]:
columns_to_round = health.columns[-3:]
health[columns_to_round] = health[columns_to_round].round(0).astype('float64')
health['LifeExpectancy'] = health['LifeExpectancy'].round(2).astype('float64')

health.head()

,GeoFIPS,GeoName,LifeExpectancy,Asthma,Diabetes,HeartDisease
0,1001,"Autauga, AL",76.13,984.0,1153.0,624.0
1,1003,"Baldwin, AL",77.92,893.0,1101.0,691.0
2,1005,"Barbour, AL",74.12,1120.0,1768.0,922.0
3,1007,"Bibb, AL",73.95,1003.0,1318.0,737.0
4,1009,"Blount, AL",76.06,987.0,1229.0,771.0


In [98]:
val_list = ['Asthma', 'Diabetes', 'HeartDisease']

for val in val_list:

    health[val] = health[val] / 100

In [99]:
health.head()

,GeoFIPS,GeoName,LifeExpectancy,Asthma,Diabetes,HeartDisease
0,1001,"Autauga, AL",76.13,9.84,11.53,6.24
1,1003,"Baldwin, AL",77.92,8.93,11.01,6.91
2,1005,"Barbour, AL",74.12,11.20,17.68,9.22
3,1007,"Bibb, AL",73.95,10.03,13.18,7.37
4,1009,"Blount, AL",76.06,9.87,12.29,7.71


In [ ]:
# dtype_mapping = {'STATE': str, 'COUNTY': str}
# urbanization = pd.read_csv(f"{grandparent_dir}/data/raw/2020_UA_COUNTY.csv", dtype=dtype_mapping)

In [2]:
industry_ts = pd.read_csv(f"{grandparent_dir}/data/raw/industry_time_series_people.csv")

industry_ts["GEO_ID"] = industry_ts["GEO_ID"].str.split("US").str[1]
industry_ts["GEO_ID"] = industry_ts["GEO_ID"].astype("int64")
industry_ts = industry_ts.rename(columns={"GEO_ID": "GeoFIPS"})

common_fips = np.intersect1d(gdp["GeoFIPS"].unique(), industry_ts["GeoFIPS"].unique())


industry_ts = industry_ts[industry_ts["GeoFIPS"].isin(common_fips)]


years = industry_ts['Year'].unique()

for year in years:
    year_df = industry_ts[industry_ts['Year'] == year]
    missing_fips = set(common_fips) - set(year_df['GeoFIPS'])
    
    if missing_fips:
        missing_data = {'Year': [year] * len(missing_fips), 'GeoFIPS': list(missing_fips)}
        
        # Fill all columns from the fourth column (index 3) onward with 0
        for col in industry_ts.columns[2:]:
            missing_data[col] = 0
        
        missing_df = pd.DataFrame(missing_data)
        industry_ts = pd.concat([industry_ts, missing_df], ignore_index=True)

industry_ts = industry_ts.merge(gdp[["GeoFIPS", "GeoName"]], on="GeoFIPS", how="left")


industry_ts = industry_ts[
    [
        "GeoFIPS",
        "GeoName",
        "Year",
        'agriculture_total',
        'mining_total',
        'construction_total',
        'manufacturing_total',
        'wholesale_trade_total',
        'retail_trade_total',
        'transportation_warehousing_total',
        'utilities_total',
        'information_total',
        'finance_insurance_total',
        'real_estate_total',
        'professional_services_total',
        'management_enterprises_total',
        'admin_support_services_total',
        'educational_services_total',
        'healthcare_social_services_total',
        'arts_recreation_total',
        'accommodation_food_services_total',
        'other_services_total',
        'public_administration_total',
    ]
]

industry_ts = industry_ts.sort_values(by=["GeoFIPS", "GeoName", "Year"])

In [5]:
missing_values = industry_ts.isnull().sum()

missing_values

GeoFIPS                              0
GeoName                              0
Year                                 0
agriculture_total                    0
mining_total                         0
construction_total                   0
manufacturing_total                  0
wholesale_trade_total                0
retail_trade_total                   0
transportation_warehousing_total     0
utilities_total                      0
information_total                    0
finance_insurance_total              0
real_estate_total                    0
professional_services_total          0
management_enterprises_total         0
admin_support_services_total         0
educational_services_total           0
healthcare_social_services_total     0
arts_recreation_total                0
accommodation_food_services_total    0
other_services_total                 0
public_administration_total          0
dtype: int64

In [115]:
columns_to_save = industry_ts.columns[industry_ts.columns.get_loc('Year') + 1:]


for column in columns_to_save:

    selected_columns = ['GeoFIPS', 'GeoName', 'Year', column]
    subsetindustry_ts = industry_ts[selected_columns]

    subsetindustry_ts.rename(columns={column: 'Value'}, inplace=True)
    

    subsetindustry_ts_long = subsetindustry_ts.copy()

    file_name_long = f"industry_{column}_long.csv"
    subsetindustry_ts_long.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_long}", index=False)



    subsetindustry_ts_std_long = standardize_and_scale(subsetindustry_ts)

    file_name_std = f"industry_{column}_std_long.csv"
    subsetindustry_ts_std_long.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_std}", index=False)


    subsetindustry_ts_wide = subsetindustry_ts.pivot_table(
    index=["GeoFIPS", "GeoName"], columns="Year", values="Value"
    )
    subsetindustry_ts_wide.reset_index(inplace=True)
    subsetindustry_ts_wide.columns.name = None

    
    file_name_wide = f"industry_{column}_wide.csv"
    subsetindustry_ts_wide.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_wide}", index=False)
    
    

    subsetindustry_ts_std_wide = standardize_and_scale(subsetindustry_ts_wide)

    assert (subsetindustry_ts_std_wide[column] >= -1).all() and subsetindustry_ts_std_wide[column] <= 1).all(), 'STD ERROR'



    file_name_std_wide = f"industry_{column}_std_wide.csv"
    subsetindustry_ts_std_wide.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_std_wide}", index=False)

    # first standardize then save

/tmp/ipykernel_13900/2843878824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsetindustry_ts.rename(columns={column: 'Value'}, inplace=True)


KeyError: 'agriculture_total'

In [ ]:
assert (data.std_wide[feature][column] >= -1).all() and (
                    data.std_wide[feature][column] <= 1
                ).all(), std_error

In [108]:
columns_to_check = industry_ts.columns[3:]

# Check for values outside the range [0, 1,000,000] in these columns
values_outside_range = industry_ts[columns_to_check].apply(lambda x: (x < 0) | (x > 1000000))

# Display rows where values are outside the range
rows_with_values_outside_range = industry_ts[values_outside_range.any(axis=1)]

In [109]:
rows_with_values_outside_range

,GeoFIPS,GeoName,Year,agriculture_total,mining_total,construction_total,manufacturing_total,wholesale_trade_total,retail_trade_total,transportation_warehousing_total,...,real_estate_total,professional_services_total,management_enterprises_total,admin_support_services_total,educational_services_total,healthcare_social_services_total,arts_recreation_total,accommodation_food_services_total,other_services_total,public_administration_total


In [4]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

loaded_data


with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

# Save the loaded data to a CSV file
#loaded_data.to_csv('your_output_file.csv', index=False) 

{'transport': array([ 2282, 15901, 22051, 22071, 22075, 22087, 22089, 22095, 22103,
        51901, 51903, 51907, 51911, 51913, 51918, 51919, 51921, 51923,
        51929, 51931, 51933, 51939, 51941, 51942, 51944, 51945, 51947,
        51949, 51951, 51953, 51955, 51958])}

In [6]:
with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)


loaded_data

{'transport': array([ 2282, 15901, 22051, 22071, 22075, 22087, 22089, 22095, 22103,
        51901, 51903, 51907, 51911, 51913, 51918, 51919, 51921, 51923,
        51929, 51931, 51933, 51939, 51941, 51942, 51944, 51945, 51947,
        51949, 51951, 51953, 51955, 51958])}

In [12]:
df = pd.DataFrame.from_dict(loaded_data)  # 'orient' specifies that the keys should be used as the index

df


melted_df = pd.melt(df, var_name='dataset', value_name='exclusions')
melted_df['dataset'] = 'transport'
melted_df

,dataset,exclusions
0,transport,2282
1,transport,15901
2,transport,22051
3,transport,22071
4,transport,22075
5,transport,22087
6,transport,22089
7,transport,22095
8,transport,22103
9,transport,51901


In [44]:
industry_ts 

,GeoFIPS,GeoName,Year,agriculture_total,mining_total,construction_total,manufacturing_total,wholesale_trade_total,retail_trade_total,transportation_warehousing_total,...,real_estate_total,professional_services_total,management_enterprises_total,admin_support_services_total,educational_services_total,healthcare_social_services_total,arts_recreation_total,accommodation_food_services_total,other_services_total,public_administration_total
1067,1001,"Autauga, AL",2010,0.006738,0.002711,0.077400,0.141284,0.031141,0.126659,0.032743,...,0.017419,0.045027,0.000246,0.024691,0.068567,0.105748,0.017419,0.069348,0.045520,0.116676
4180,1001,"Autauga, AL",2011,0.010150,0.002320,0.069804,0.139650,0.036290,0.124984,0.042255,...,0.016902,0.043042,0.000000,0.027466,0.075314,0.107834,0.016778,0.064833,0.044285,0.112722
9238,1001,"Autauga, AL",2012,0.010860,0.003482,0.056456,0.135876,0.029720,0.134964,0.044933,...,0.016456,0.043026,0.000000,0.031171,0.072746,0.115233,0.019150,0.058860,0.044684,0.114031
10327,1001,"Autauga, AL",2013,0.010193,0.003843,0.058027,0.128838,0.030163,0.135272,0.048502,...,0.015666,0.046831,0.000000,0.031374,0.063333,0.113757,0.019092,0.056523,0.059114,0.113924
13388,1001,"Autauga, AL",2014,0.011349,0.004391,0.056124,0.132295,0.031976,0.127863,0.040881,...,0.015118,0.049704,0.000000,0.031852,0.063041,0.115934,0.017686,0.058402,0.061550,0.112993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23555,56045,"Weston, WY",2017,0.053713,0.192545,0.052245,0.061638,0.012328,0.112122,0.061344,...,0.002348,0.011154,0.000000,0.017317,0.125037,0.093044,0.005283,0.056648,0.025829,0.060757
25620,56045,"Weston, WY",2018,0.048934,0.203890,0.065558,0.050816,0.012861,0.094417,0.073714,...,0.001882,0.016311,0.000000,0.018193,0.115747,0.091280,0.008469,0.053011,0.021330,0.057403
28703,56045,"Weston, WY",2019,0.062093,0.188140,0.049674,0.054331,0.012729,0.093760,0.064266,...,0.004346,0.025147,0.000000,0.022043,0.113940,0.081341,0.008072,0.054020,0.049053,0.064887
32913,56045,"Weston, WY",2020,0.046048,0.163918,0.072509,0.069759,0.010309,0.100344,0.048454,...,0.020619,0.023368,0.000000,0.021649,0.079725,0.101375,0.018557,0.029897,0.053952,0.049485
